### LANGCHAIN
参考： https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

IMPORT LIBRARY

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4 pypdf

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

UPLOAD FILE

https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/

In [ ]:
# # WEB URL
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )
# docs = loader.load()

# print(docs[0].page_content[:500])

In [ ]:
# PDF
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/Final Project Report.pdf")
docs = loader.load()
# pages = loader.load_and_split()
print(docs[0].page_content[:500])

Neural Style Transfer Final Project Report for CS-GY 6923_1_INET Machine Learning Yunge Wen, yw3776  Table of Content - Introduction - Convolutional Neural Network - VGG-19 Architecture & Feature Extraction - Neural Style Transfer  - Content Loss  - Gram Matrix & Style Loss  - Optimization - Next Steps: Vision Transformer and Segment Anything Model - Citations  Introduction  The groundbreaking research by Gatys et al. showcased the capabilities of Convolutional Neural Networks (CNNs) in generati


In [ ]:
# # TXT
# from langchain_community.document_loaders import TextLoader

# loader = TextLoader("/content/sample_data/README.md")
# docs = loader.load()

# print(docs[0].page_content[:500])

SPLIT TEXT

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(docs)

print(splits[0].page_content)

Neural Style Transfer Final Project Report for CS-GY 6923_1_INET Machine Learning Yunge Wen, yw3776  Table of Content - Introduction - Convolutional Neural Network - VGG-19 Architecture & Feature Extraction - Neural Style Transfer  - Content Loss  - Gram Matrix & Style Loss  - Optimization - Next Steps: Vision Transformer and Segment Anything Model - Citations  Introduction  The groundbreaking research by Gatys et al. showcased the capabilities of Convolutional Neural Networks (CNNs) in generating artistic images by disentangling and recombining image content and style. This technique, known as Neural Style Transfer (NST), has since garnered significant attention in both academic circles and industrial applications.  In this project, I reproduced the seminal paper by visualizing the hidden layers of content and style feature maps within the neural network architecture step by step. Additionally, I propose leveraging NST in combination with the latest Segment Anything Model to achieve


VECTOR EMBEDDING

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

RETRIEVAL ONLY

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What is CS-GY 6923_1_INET?")
print(retrieved_docs[0].page_content)

Neural Style Transfer Final Project Report for CS-GY 6923_1_INET Machine Learning Yunge Wen, yw3776  Table of Content - Introduction - Convolutional Neural Network - VGG-19 Architecture & Feature Extraction - Neural Style Transfer  - Content Loss  - Gram Matrix & Style Loss  - Optimization - Next Steps: Vision Transformer and Segment Anything Model - Citations  Introduction  The groundbreaking research by Gatys et al. showcased the capabilities of Convolutional Neural Networks (CNNs) in generating artistic images by disentangling and recombining image content and style. This technique, known as Neural Style Transfer (NST), has since garnered significant attention in both academic circles and industrial applications.  In this project, I reproduced the seminal paper by visualizing the hidden layers of content and style feature maps within the neural network architecture step by step. Additionally, I propose leveraging NST in combination with the latest Segment Anything Model to achieve


PROMPTING

In [ ]:
# PROMPT

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

example_messages = custom_rag_prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

Helpful Answer:


GENERATE

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Who is Yunge?"):
    print(chunk, end="", flush=True)

Yunge Wen is the individual who authored the Neural Style Transfer Final Project Report for CS-GY 6923_1_INET Machine Learning, where they explored the capabilities of Convolutional Neural Networks in generating artistic images through disentangling and recombining image content and style. Yunge Wen is a student who reproduced the seminal paper by visualizing hidden layers of content and style feature maps within the neural network architecture and proposed leveraging Neural Style Transfer with the Segment Anything Model for enhanced visual effects.

In [ ]:
for chunk in rag_chain.stream("What is CS-GY 6923_1_INET?"):
    print(chunk, end="", flush=True)

CS-GY 6923_1_INET is a Machine Learning course that covers topics such as Convolutional Neural Networks, VGG-19 Architecture, Feature Extraction, and Neural Style Transfer. The course involves reproducing seminal papers in the field and exploring advanced techniques like leveraging Neural Style Transfer with the Segment Anything Model. It combines elements of art and research, making it a unique and interdisciplinary learning experience.

In [ ]:
!pip install gradio

In [ ]:
# 官方 upload button

import gradio as gr

def upload_file(files):
    file_paths = [file.name for file in files]
    return file_paths

with gr.Blocks() as demo:
    file_output = gr.File()
    upload_button = gr.UploadButton("Click to Upload a File", file_types=["image", "video"], file_count="multiple")
    upload_button.upload(upload_file, upload_button, file_output)

demo.launch()

In [ ]:
# https://blog.csdn.net/qq_51116518/article/details/132628392

import os
import gradio as gr
import tempfile
import shutil

def generate_file(file_obj):
    global tmpdir
    print('临时文件夹地址：{}'.format(tmpdir))
    print('上传文件的地址：{}'.format(file_obj.name)) # 输出上传后的文件在gradio中保存的绝对地址

    #获取到上传后的文件的绝对路径后，其余的操作就和平常一致了

    shutil.copy(file_obj.name, tmpdir)    # 将文件复制到临时目录中
    FileName=os.path.basename(file_obj.name)     # 获取上传Gradio的文件名称
    NewfilePath=os.path.join(tmpdir,FileName)     # 获取拷贝在临时目录的新的文件地址
    print(NewfilePath)

    # 打开复制到新路径后的文件
    with open(NewfilePath, 'rb') as file_obj:

        #在本地电脑打开一个新的文件，并且将上传文件内容写入到新文件
        outputPath=os.path.join(tmpdir,"New"+FileName)
        with open(outputPath,'wb') as w:
            w.write(file_obj.read())

    # 返回新文件的的地址（注意这里）
    return outputPath

def main():
    global tmpdir
    with tempfile.TemporaryDirectory(dir='.') as tmpdir:

        inputs = gr.components.File(label="上传文件")
        outputs = gr.components.File(label="下载文件")

        app = gr.Interface(
            fn=generate_file,
            inputs=inputs,
            outputs=outputs,
            title="文件上传、并生成可下载文件demo",
            description="上传任何文件都可以，只要大小别超过你电脑的内存即可"
      )

        app.launch(share=True)

if __name__=="__main__":
    main()



In [ ]:
import os
import gradio as gr
import tempfile
import shutil

file_obj = '/content/sample_data/README.md'

global tmpdir
print('临时文件夹地址：{}'.format(tmpdir))
print('上传文件的地址：{}'.format(file_obj.name)) # 输出上传后的文件在gradio中保存的绝对地址

In [ ]:
app = gr.Interface(
    fn = None,
    inputs = gr.components.File(label="上传文件"),
    outputs = None,
    gr.components.Textbox()
)

app.launch()

In [ ]:
gradio.Textbox(···)